# Setup
This IPython notebook will walk through the steps of characterizing iModulons through the semi-automated tools in PyModulon. You will need:

* M and A matrices
* Expression data (e.g. `log_tpm_norm.csv`)
* Gene table and KEGG/GO annotations (Generated in `gene_annotation.ipynb`)
* Sample table, with a column for `project` and `condition`
* TRN file

Optional:
* iModulon table (if you already have some characterized iModulons)

In [1]:
from pymodulon.core import IcaData
from pymodulon.plotting import *
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = path.join('..','data','5g_processed_data')
ica_data_dir = path.join('..','data','5g_interim')

# GO and KEGG annotations are in the 'external' folder
external_data = path.join('..','data','external')

## Check your sample table (i.e. metadata file)
Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'final_meta.tsv'),sep='\t')
df_metadata.head()

,exp_theme,exp_condition,sample,shortd,longd,carbon_source,oxygen_level,nitrate_level,copper_level,lanthanum_level,growth_rate,growth_mode,include?,notes,cluster_id,project,condition,reference_condition
0,uMax,uMax,5GB1_FM03_TR1_QC,uMax_QC,"Fermentor run 3, uMax though close to O2 limit...",2,1,0,3,0,2,0,1,NaN,1,5G,uMax,uMax
1,uMax,uMax,5GB1_FM03_TR2_QC,uMax_QC,"Fermentor run 3, uMax though close to O2 limit...",2,1,0,3,0,2,0,1,NaN,3,5G,uMax,uMax
2,lowO2_fast_growth,lowO2_fast_growth,5GB1_FM11_TR1_QC,lowO2_QC,"Fermentor run 11, O2 limited, QC",2,0,0,3,0,1,0,1,NaN,18,5G,lowO2_fast_growth,uMax
3,lowO2_fast_growth,lowO2_fast_growth,5GB1_FM11_TR2_QC,lowO2_QC,"Fermentor run 11, O2 limited, QC",2,0,0,3,0,1,0,1,NaN,18,5G,lowO2_fast_growth,uMax
4,lowCH4,lowCH4,5GB1_FM12_TR1,lowCH4,"Fermentor run 12, methane limited",1,1,0,3,0,1,0,1,NaN,17,5G,lowCH4,uMax


In [4]:
df_metadata.columns

Index(['exp_theme', 'exp_condition', 'sample', 'shortd', 'longd',
       'carbon_source', 'oxygen_level', 'nitrate_level', 'copper_level',
       'lanthanum_level', 'growth_rate', 'growth_mode', 'include?', 'notes',
       'cluster_id', 'project', 'condition', 'reference_condition'],
      dtype='object')

In [5]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## ~Check your TRN~ - We have no TRN for 5G!

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [8]:
# df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))
# df_trn.head()

In [10]:
#df_trn[df_trn['regulator'] == 'AbrB']

,regulator,gene_id,effect
4683,AbrB,BSU_24640,repression
4684,AbrB,BSU_14890,repression
4685,AbrB,BSU_34390,repression
4686,AbrB,BSU_40180,repression
4687,AbrB,BSU_02330,repression
...,...,...,...
4949,AbrB,BSU_33410,repression
4950,AbrB,BSU_18910,repression
4951,AbrB,BSU_01960,repression
4952,AbrB,BSU_06240,repression


The `regulator` and `gene_id` must be filled in for each row

In [11]:
# print(df_trn.regulator.notnull().all())
# print(df_trn.gene_id.notnull().all())

True
True


## Load the data
You're now ready to load your IcaData object!

In [13]:
m = pd.read_csv(path.join(ica_data_dir,'M.csv'))
m.head()

,locus_tag,0,1,2,3,4,5,6,7,8,...,43,44,45,46,47,48,49,50,51,52
0,EQU24_RS00005,-0.001218,-0.012210,-0.000148,0.001630,-0.007975,-0.002027,0.001659,-0.002002,-0.001159,...,0.007620,-0.017961,-0.003422,-0.005579,0.000207,-0.001315,-0.000140,-0.001118,0.005176,-0.000223
1,EQU24_RS00010,-0.000421,-0.016359,-0.008427,0.006276,-0.007272,-0.002388,-0.006465,0.002140,0.002844,...,0.008849,-0.003400,-0.003843,-0.008979,0.004835,0.006598,0.001099,-0.008522,0.007841,0.000336
2,EQU24_RS00015,0.014323,-0.004434,0.001522,0.014076,-0.004344,-0.002314,-0.006279,0.011183,-0.002367,...,0.000502,-0.010556,-0.001445,-0.004536,0.008162,-0.001967,-0.000995,0.006729,-0.001812,0.001697
3,EQU24_RS00020,0.003416,0.001109,0.008821,0.000094,0.000696,0.000544,-0.002481,0.008632,0.000616,...,0.003644,-0.004934,-0.003466,0.000330,-0.003172,0.004194,0.005029,-0.003325,-0.002615,0.007750
4,EQU24_RS00025,0.024772,0.008447,0.006155,0.000959,-0.004074,0.000910,-0.005474,-0.002638,0.002965,...,-0.006944,0.020871,0.001575,-0.003430,-0.007034,0.008026,0.016603,-0.004422,0.008380,0.011851


In [14]:
a = pd.read_csv(path.join(ica_data_dir,'A.csv'))
a.head()

,Unnamed: 0,5GB1_FM03_TR1_QC_tpm,5GB1_FM03_TR2_QC_tpm,5GB1_FM11_TR1_QC_tpm,5GB1_FM11_TR2_QC_tpm,5GB1_FM12_TR1_tpm,5GB1_FM12_TR1_QC_tpm,5GB1_FM12_TR2_tpm,5GB1_FM12_TR2_QC_tpm,5GB1_FM14_TR1_tpm,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,0,9.529850,7.499091,5.468450,5.010820,-5.870309,-5.777901,-10.125820,-10.326414,-1.335729,...,3.508544,3.917175,-1.753262,-3.977450,-3.956464,-4.478062,-0.741869,-1.859980,-2.955320,-3.948350
1,1,0.332269,-0.736224,1.015687,-0.382087,0.998179,1.016328,1.840793,1.650056,-0.647088,...,-2.291153,-2.547193,-11.787137,-10.148064,-12.942754,-5.250938,-8.754121,-12.819043,-4.555001,-16.266555
2,2,0.358727,0.723006,-0.660407,0.284854,-14.235708,-14.269141,-14.105268,-14.393936,-13.349594,...,-6.672029,-5.477086,4.368587,4.047420,4.256874,3.606128,4.391246,4.378823,4.961652,3.977729
3,3,-5.659867,1.427957,2.188699,1.240829,1.770186,1.287521,1.572553,1.125283,1.544704,...,3.870540,2.919166,3.094606,2.251679,3.030040,0.074859,3.538138,3.847606,3.462035,2.261120
4,4,-8.099084,-1.329372,1.478984,0.427919,0.733313,0.889191,0.728142,1.879416,1.258482,...,0.898605,1.313268,4.971210,4.746343,4.872671,5.165037,4.982777,4.976336,5.840944,4.804737


In [16]:
x = pd.read_csv(path.join(data_dir,'5g_log_tpm_norm_indiv.csv'))
x.head()

,locus_tag,5GB1_FM03_TR1_QC_tpm,5GB1_FM03_TR2_QC_tpm,5GB1_FM11_TR1_QC_tpm,5GB1_FM11_TR2_QC_tpm,5GB1_FM12_TR1_tpm,5GB1_FM12_TR1_QC_tpm,5GB1_FM12_TR2_tpm,5GB1_FM12_TR2_QC_tpm,5GB1_FM14_TR1_tpm,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,EQU24_RS00005,-0.081618,0.013543,-1.005473,-0.565908,-0.868240,-0.861181,-0.860286,-0.810841,-0.466774,...,-0.423331,-0.413887,-0.465584,-0.361508,-0.366746,-0.629039,-0.374679,-0.430905,-0.773416,-0.326023
1,EQU24_RS00010,-0.213951,-0.115316,-0.677320,-0.569877,-1.119576,-1.054664,-1.167149,-1.149678,-0.423026,...,-0.179590,-0.182090,-0.479849,-0.274384,-0.346861,-0.306024,-0.388553,-0.356148,-0.679766,-0.170025
2,EQU24_RS00015,0.002850,-0.174656,-0.639651,-0.279816,-1.029608,-1.120098,-1.028006,-0.996256,-0.586234,...,-0.110156,0.045097,-0.655508,-0.483245,-0.609796,-0.620538,-0.578460,-0.572416,-0.801799,-0.181680
3,EQU24_RS00020,-0.130356,0.040745,-0.414605,-0.346534,-0.941858,-0.957353,-0.913985,-0.980052,-0.512366,...,0.082403,0.193677,-0.363136,-0.123069,-0.244925,-0.128927,-0.032470,-0.106898,-0.295688,-0.049523
4,EQU24_RS00025,0.659048,0.404700,-0.031390,0.291903,-0.787794,-0.640604,-0.818610,-0.839456,-0.307450,...,0.087352,0.059120,-0.455537,-0.375085,-0.325837,-0.267183,-0.280655,-0.400990,-0.192058,-0.235679


In [11]:
g = pd.read_csv(path.join(data_dir,'final_meta.tsv'),sep='\t',index_col=0)
g.tail()

,exp_theme,exp_condition,sample,shortd,longd,carbon_source,oxygen_level,nitrate_level,copper_level,lanthanum_level,growth_rate,growth_mode,include?,notes,cluster_id,project,condition,reference_condition
sample_id,,,,,,,,,,,,,,,,,,
5GB1C-5G-N-BR2_tpm,Lanthanum,NoLanthanum,5GB1C-5G-N-BR2,NoLa_rep2,Vial sample in mid- to late-exponential phase,2,1,0,3,0,2,1,1,NaN,12,5G,NoLanthanum,uMax
5GB1C-JG15-La-BR1_tpm,Lanthanum,WithLanthanum,5GB1C-JG15-La-BR1,deltaTBDT_La_rep1,Vial sample in mid- to late-exponential phase,2,1,0,3,1,2,1,1,NaN,12,5G,WithLanthanum,uMax
5GB1C-JG15-La-BR2_tpm,Lanthanum,WithLanthanum,5GB1C-JG15-La-BR2,deltaTBDT_La_rep2,Vial sample in mid- to late-exponential phase,2,1,0,3,1,2,1,1,NaN,12,5G,WithLanthanum,uMax
5GB1C-JG15-N-BR1_tpm,Lanthanum,NoLanthanum,5GB1C-JG15-N-BR1,deltaTBDT_NoLa_rep1,Vial sample in mid- to late-exponential phase,2,1,0,3,0,2,1,1,NaN,12,5G,NoLanthanum,uMax
5GB1C-JG15-N-BR2_tpm,Lanthanum,NoLanthanum,5GB1C-JG15-N-BR2,deltaTBDT_NoLa_rep2,Vial sample in mid- to late-exponential phase,2,1,0,3,0,2,1,1,NaN,12,5G,NoLanthanum,uMax


In [8]:
s = pd.read_csv(path.join(data_dir,'5G_gene_info2.csv'))
s.tail()

,locus_tag,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,gene_len,tu_name,tu_id,gene_name_extra,ncbi_protein,strand,COG,uniprot
4208,EQU24_RS22135,tRNA uridine-5-carboxymethylaminomethyl(34) sy...,CDS,mnmE,NZ_CP035467.1,4994941,4996288,Derived by automated computational analysis us...,VDIGTNDTIAAIATPPGNGGVGIVRISGPAVSIIAAKLTDRPLPPR...,1348,rpmH // rnpA // yidD // yidC // mnmE,1,mnmE||mnmE,WP_017841475.1,-1,J,A0A4P9USW8
4209,EQU24_RS22140,membrane protein insertase YidC,CDS,yidC,NZ_CP035467.1,4996295,4997993,Derived by automated computational analysis us...,MDNIRFILVVTFAMLLYMLYDAWQIDYGPKREISVAQQMPQDLKED...,1699,rpmH // rnpA // yidD // yidC // mnmE,1,yidC||yidC,WP_017841474.1,-1,U,A0A4V1IKE6
4210,EQU24_RS22145,membrane protein insertion efficiency factor YidD,CDS,yidD,NZ_CP035467.1,4997998,4998220,Derived by automated computational analysis us...,MRVLLIAIIKLYQYFISPLLGKNCRFYPSCSCYALEALHKHGAAQG...,223,rpmH // rnpA // yidD // yidC // mnmE,1,yidD||yidD,WP_083877753.1,-1,S,UPI000A004C04
4211,EQU24_RS22150,ribonuclease P protein component,CDS,rnpA,NZ_CP035467.1,4998201,4998570,Derived by automated computational analysis us...,LTNKVFSFPPQLRLRKPSEYKKVFTGPVKSSDAYFTLLAVRNELDH...,370,rpmH // rnpA // yidD // yidC // mnmE,1,rnpA||rnpA,WP_017841473.1,-1,J,A0A4P9UT31
4212,EQU24_RS22155,50S ribosomal protein L34,CDS,rpmH,NZ_CP035467.1,4998574,4998709,Derived by automated computational analysis us...,MKRTYQPSKIKRVRTHGFRARMATKGGRKVLNARRAKGRAKLTV*,136,rpmH // rnpA // yidD // yidC // mnmE,1,rpmH||rpmH,WP_083877752.1,-1,J,UPI00034B9E08


In [12]:
# ERIN: change this to the new M and A?
# ica_data = IcaData(M = path.join(ica_data_dir,'M.csv'),
#                    A = path.join(ica_data_dir,'A.csv'),
#                    X = path.join(data_dir,'5g_log_tpm_norm_indiv.csv'),
#                    gene_table = path.join(data_dir,'5G_gene_info2.csv'),
#                    sample_table = path.join(data_dir,'final_meta.tsv'),
#                    #trn = path.join(external_data,'TRN.csv'),
#                    optimize_cutoff=True)

If you don't have a TRN (or have a very minimal TRN), use `threshold_method = 'kmeans'`

In [13]:
# ERIN: ah, I'll use this version for 5g? bc no TRN

ica_data = IcaData(M = path.join(ica_data_dir,'M.csv'),
                   A = path.join(ica_data_dir,'A.csv'),
                   X = path.join(data_dir,'5g_log_tpm_norm_indiv.csv'),
                   gene_table = path.join(data_dir,'5G_gene_info2.csv'),
                   sample_table = path.join(data_dir,'final_meta.tsv'),
                   #trn = path.join(data_dir,'TRN.csv'),
                   threshold_method = 'kmeans')

In [16]:
ica_data.A

,5GB1_FM03_TR1_QC_tpm,5GB1_FM03_TR2_QC_tpm,5GB1_FM11_TR1_QC_tpm,5GB1_FM11_TR2_QC_tpm,5GB1_FM12_TR1_tpm,5GB1_FM12_TR1_QC_tpm,5GB1_FM12_TR2_tpm,5GB1_FM12_TR2_QC_tpm,5GB1_FM14_TR1_tpm,5GB1_FM14_TR1_QC_tpm,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,9.529850,7.499091,5.468450,5.010820,-5.870309,-5.777901,-10.125820,-10.326414,-1.335729,-0.890927,...,3.508544,3.917175,-1.753262,-3.977450,-3.956464,-4.478062,-0.741869,-1.859980,-2.955320,-3.948350
1,0.332269,-0.736224,1.015687,-0.382087,0.998179,1.016328,1.840793,1.650056,-0.647088,-0.897334,...,-2.291153,-2.547193,-11.787137,-10.148064,-12.942754,-5.250938,-8.754121,-12.819043,-4.555001,-16.266555
2,0.358727,0.723006,-0.660407,0.284854,-14.235708,-14.269141,-14.105268,-14.393936,-13.349594,-13.632557,...,-6.672029,-5.477086,4.368587,4.047420,4.256874,3.606128,4.391246,4.378823,4.961652,3.977729
3,-5.659867,1.427957,2.188699,1.240829,1.770186,1.287521,1.572553,1.125283,1.544704,1.784598,...,3.870540,2.919166,3.094606,2.251679,3.030040,0.074859,3.538138,3.847606,3.462035,2.261120
4,-8.099084,-1.329372,1.478984,0.427919,0.733313,0.889191,0.728142,1.879416,1.258482,1.004572,...,0.898605,1.313268,4.971210,4.746343,4.872671,5.165037,4.982777,4.976336,5.840944,4.804737
5,5.379447,6.103335,5.971049,6.755312,6.570261,7.135628,7.258388,7.121024,6.088885,5.879550,...,7.439557,7.413246,10.511500,9.969951,9.783052,9.800620,10.655251,11.031199,10.546883,9.227699
6,-34.612398,4.646587,2.122133,2.875662,3.455195,3.403894,3.365507,4.287781,2.728459,3.575446,...,4.492622,3.823522,7.196000,6.271132,7.270290,5.919232,8.376768,8.400545,8.013571,6.389559
7,1.036241,0.192585,1.960135,1.434704,1.402801,1.816734,1.065198,1.422298,0.817771,0.768796,...,2.209515,2.620932,5.594312,6.999508,0.832677,1.918987,17.433299,9.687344,4.822446,18.178736
8,-0.063541,-0.119997,-0.585893,-0.078129,-0.395151,-0.562576,-0.327792,-0.398948,0.475446,0.557561,...,-31.288591,-29.762727,-29.926512,-30.565501,-1.248837,0.378603,-1.674328,-1.506295,-0.739802,-1.149169
9,-0.270272,-0.252498,-0.607710,-0.298202,-7.198589,-7.287829,-7.049716,-7.348552,-4.857358,-4.957782,...,-4.796641,-4.763849,-8.941949,-8.763670,-9.286260,-7.532056,-8.005278,-9.266830,-9.896174,-9.343441


# Regulatory iModulons
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

In [17]:
#ica_data.compute_trn_enrichment()

You can also search for AND/OR combinations of regulators using the `max_regs` argument.

Regulator enrichments can be directly saved to the `imodulon_table` using the `save` argument. This saves the enrichment with the lowest q-value to the table.

In [22]:
# First search for regulator enrichments with 2 regulators
# ica_data.compute_trn_enrichment(max_regs=2,save=True)

# # Next, search for regulator enrichments with just one regulator. This will supercede the 2 regulator enrichments.
# ica_data.compute_trn_enrichment(max_regs=1,save=True)

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,0,S-box,1.436720e-30,8.620321e-30,0.652174,0.576923,0.612245,15.0,26.0,23.0,1.0
1,0,ThrR,5.239175e-07,1.571752e-06,0.130435,0.750000,0.222222,3.0,4.0,23.0,1.0
2,2,WalR,1.897147e-12,2.656006e-11,0.388889,0.291667,0.333333,7.0,24.0,18.0,1.0
3,2,RemA,8.460083e-12,5.922058e-11,0.388889,0.241379,0.297872,7.0,29.0,18.0,1.0
4,2,OpcR,1.456062e-08,6.794954e-08,0.222222,0.500000,0.307692,4.0,8.0,18.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
126,66,AbrB,2.100153e-07,7.875574e-07,0.375000,0.044280,0.079208,12.0,271.0,32.0,1.0
127,66,IolR,1.685319e-06,4.213298e-06,0.125000,0.307692,0.177778,4.0,13.0,32.0,1.0
128,66,LicR,1.465024e-06,4.213298e-06,0.093750,0.750000,0.166667,3.0,4.0,32.0,1.0
129,67,KipR,6.440803e-21,3.220402e-20,0.777778,1.000000,0.875000,7.0,7.0,9.0,1.0


The list of regulatory iModulons are shown below

In [23]:
# regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]
# print(len(ica_data.imodulon_table),'Total iModulons')
# print(len(regulatory_imodulons),'Regulatory iModulons')
# regulatory_imodulons

72 Total iModulons
55 Regulatory iModulons


,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,S-box,1.436720e-30,8.620321e-30,0.652174,0.576923,0.612245,15.0,26.0,23.0,1.0
2,WalR,1.897147e-12,2.656006e-11,0.388889,0.291667,0.333333,7.0,24.0,18.0,1.0
4,Rex,1.416977e-19,7.084883e-19,1.000000,0.583333,0.736842,7.0,12.0,7.0,1.0
5,Fnr,1.066741e-14,6.400444e-14,0.600000,0.545455,0.571429,6.0,11.0,10.0,1.0
8,CsoR/Fnr,1.842216e-11,8.799896e-07,0.113636,0.400000,0.176991,10.0,25.0,88.0,2.0
10,SigM,1.614855e-27,2.099311e-26,0.894737,0.182796,0.303571,17.0,93.0,19.0,1.0
11,EAR riboswitch,3.154667e-40,2.208267e-39,0.833333,1.000000,0.909091,15.0,15.0,18.0,1.0
12,SigB,4.367180e-66,6.987488e-65,0.962264,0.230769,0.372263,51.0,221.0,53.0,1.0
13,PyrR,6.910434e-27,1.382087e-26,1.000000,0.900000,0.947368,9.0,10.0,9.0,1.0
14,SigV,2.324641e-12,3.022033e-11,0.333333,0.333333,0.333333,7.0,21.0,21.0,1.0


You can rename iModulons in this jupyter notebook, or you can save the iModulon table as a CSV and edit it in Excel.

If two iModulons have the same regulator (e.g. 'Reg'), they will be named 'Reg-1' and 'Reg-2'

In [24]:
# ica_data.rename_imodulons(regulatory_imodulons.regulator.to_dict())
# ica_data.imodulon_table.head()

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
S-box,S-box,1.436720e-30,8.620321e-30,0.652174,0.576923,0.612245,15.0,26.0,23.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WalR,WalR,1.897147e-12,2.656006e-11,0.388889,0.291667,0.333333,7.0,24.0,18.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rex,Rex,1.416977e-19,7.084883e-19,1.000000,0.583333,0.736842,7.0,12.0,7.0,1.0


In [25]:
# regulatory_imodulons = ica_data.imodulon_table[ica_data.imodulon_table.regulator.notnull()]

# Functional iModulons

GO annotations and KEGG pathways/modules were generated in the 1_create_the_gene_table.ipynb notebook. Enrichments will be calculated in this notebook, and further curated in the 3_manual_iModulon_curation notebook.

## GO Enrichments

First load the Gene Ontology annotations

In [20]:
DF_GO = pd.read_csv(path.join(external_data,'5G_GO_annotations_from_egg.csv'),index_col=0)
DF_GO.head()

,locus_tag,go_id,go_name
0,EQU24_RS00005,GO:0000166,nucleotide binding
1,EQU24_RS00005,GO:0003674,molecular_function
2,EQU24_RS00005,GO:0003676,nucleic acid binding
3,EQU24_RS00005,GO:0003677,DNA binding
4,EQU24_RS00005,GO:0003688,DNA replication origin binding


In [32]:
DF_GO['gene_id'] = DF_GO['locus_tag']
DF_GO.head(15)

,locus_tag,go_id,go_name,gene_id
0,EQU24_RS00005,GO:0000166,nucleotide binding,EQU24_RS00005
1,EQU24_RS00005,GO:0003674,molecular_function,EQU24_RS00005
2,EQU24_RS00005,GO:0003676,nucleic acid binding,EQU24_RS00005
3,EQU24_RS00005,GO:0003677,DNA binding,EQU24_RS00005
4,EQU24_RS00005,GO:0003688,DNA replication origin binding,EQU24_RS00005
5,EQU24_RS00005,GO:0003690,double-stranded DNA binding,EQU24_RS00005
6,EQU24_RS00005,GO:0005488,binding,EQU24_RS00005
7,EQU24_RS00005,GO:0005515,protein binding,EQU24_RS00005
8,EQU24_RS00005,GO:0005524,ATP binding,EQU24_RS00005
9,EQU24_RS00005,GO:0005575,cellular_component,EQU24_RS00005


In [33]:
DF_GO_enrich = ica_data.compute_annotation_enrichment(DF_GO,'go_name')

In [37]:
DF_GO_enrich['imodulon'].value_counts()

41    245
9     106
39     47
27     10
2       4
Name: imodulon, dtype: int64

In [43]:
DF_GO_enrich[DF_GO_enrich['imodulon'] == 41]

,imodulon,go_name,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
167,41,ribosomal subunit,5.238442e-32,6.092731e-29,0.130584,0.678571,0.219020,38.0,56.0,291.0
168,41,translation,4.368441e-32,6.092731e-29,0.164948,0.510638,0.249351,48.0,94.0,291.0
169,41,structural constituent of ribosome,6.782891e-32,6.092731e-29,0.127148,0.698113,0.215116,37.0,53.0,291.0
170,41,cytosolic ribosome,5.238442e-32,6.092731e-29,0.130584,0.678571,0.219020,38.0,56.0,291.0
171,41,ribosome,1.478422e-31,1.062394e-28,0.130584,0.666667,0.218391,38.0,57.0,291.0
...,...,...,...,...,...,...,...,...,...,...
407,41,negative regulation of translational initiation,5.576652e-03,8.245643e-02,0.010309,0.500000,0.020202,3.0,6.0,291.0
408,41,phosphorylation,5.567669e-03,8.245643e-02,0.030928,0.183673,0.052941,9.0,49.0,291.0
409,41,carbohydrate derivative binding,5.549636e-03,8.245643e-02,0.044674,0.151163,0.068966,13.0,86.0,291.0
410,41,primary active transmembrane transporter activity,5.953820e-03,8.731460e-02,0.020619,0.240000,0.037975,6.0,25.0,291.0


In [38]:
ica_data.imodulon_names

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52]

In [36]:
DF_GO_enrich.sort_values('pvalue')#.head()

,imodulon,go_name,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
168,41,translation,4.368441e-32,6.092731e-29,0.164948,0.510638,0.249351,48.0,94.0,291.0
170,41,cytosolic ribosome,5.238442e-32,6.092731e-29,0.130584,0.678571,0.219020,38.0,56.0,291.0
167,41,ribosomal subunit,5.238442e-32,6.092731e-29,0.130584,0.678571,0.219020,38.0,56.0,291.0
169,41,structural constituent of ribosome,6.782891e-32,6.092731e-29,0.127148,0.698113,0.215116,37.0,53.0,291.0
171,41,ribosome,1.478422e-31,1.062394e-28,0.130584,0.666667,0.218391,38.0,57.0,291.0
...,...,...,...,...,...,...,...,...,...,...
405,41,chromosome segregation,5.576652e-03,8.245643e-02,0.010309,0.500000,0.020202,3.0,6.0,291.0
406,41,large ribosomal subunit rRNA binding,5.576652e-03,8.245643e-02,0.010309,0.500000,0.020202,3.0,6.0,291.0
407,41,negative regulation of translational initiation,5.576652e-03,8.245643e-02,0.010309,0.500000,0.020202,3.0,6.0,291.0
410,41,primary active transmembrane transporter activity,5.953820e-03,8.731460e-02,0.020619,0.240000,0.037975,6.0,25.0,291.0


## KEGG Enrichments

### Load KEGG mapping
The `kegg_mapping.csv` file contains KEGG orthologies, pathways, modules, and reactions. Only pathways and modules are relevant to iModulon characterization.

In [12]:
DF_KEGG = pd.read_csv(path.join(external_data,'kegg_mapping.csv'),index_col=0)
print(DF_KEGG.database.unique())
DF_KEGG.head()

['KEGG_pathway' 'KEGG_module' 'KEGG_reaction']


,gene_id,database,kegg_id
2553,BSU_00010,KEGG_pathway,map02020
2554,BSU_00010,KEGG_pathway,map04112
2561,BSU_00020,KEGG_pathway,map00230
2562,BSU_00020,KEGG_pathway,map00240
2563,BSU_00020,KEGG_pathway,map01100


In [13]:
kegg_pathways = DF_KEGG[DF_KEGG.database == 'KEGG_pathway']
kegg_modules = DF_KEGG[DF_KEGG.database == 'KEGG_module']

### Perform enrichment
Uses the `compute_annotation_enrichment` function

In [14]:
DF_pathway_enrich = ica_data.compute_annotation_enrichment(kegg_pathways,'kegg_id')
DF_module_enrich = ica_data.compute_annotation_enrichment(kegg_modules,'kegg_id')

In [15]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,5,map02010,1.081482e-08,2.433335e-06,0.500000,0.056738,0.101911,8.0,141.0,16.0
1,5,map00920,4.686248e-07,5.272029e-05,0.250000,0.210526,0.228571,4.0,19.0,16.0
2,6,map00730,1.308624e-13,2.944404e-11,0.466667,0.333333,0.388889,7.0,21.0,15.0
3,6,map01100,4.889884e-04,5.501119e-02,0.533333,0.012678,0.024768,8.0,631.0,15.0
4,8,map00270,3.797989e-22,8.545475e-20,0.428571,0.333333,0.375000,15.0,45.0,35.0


In [16]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size
0,1,M00583,3.639319e-04,9.789769e-02,0.024096,1.000,0.047059,2.0,2.0,83.0
1,5,M00585,3.471665e-13,9.338779e-11,0.312500,1.000,0.476190,5.0,5.0,16.0
2,5,M00436,4.156064e-08,5.589906e-06,0.187500,1.000,0.315789,3.0,3.0,16.0
3,6,M00582,1.871400e-06,5.034067e-04,0.200000,0.375,0.260870,3.0,8.0,15.0
4,6,M00127,1.116177e-04,1.501258e-02,0.133333,0.400,0.200000,2.0,5.0,15.0


### Convert KEGG IDs to human-readable names

In [17]:
for idx,key in tqdm(DF_pathway_enrich.kegg_id.items(),total=len(DF_pathway_enrich)):
    text = REST.kegg_find('pathway',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_pathway_enrich.loc[idx,'pathway_name'] = name
    except AttributeError:
        DF_pathway_enrich.loc[idx,'pathway_name'] = None
    
for idx,key in tqdm(DF_module_enrich.kegg_id.items(),total=len(DF_module_enrich)):
    text = REST.kegg_find('module',key).read()
    try:
        name = re.search('\t(.*)\n',text).group(1)
        DF_module_enrich.loc[idx,'module_name'] = name
    except AttributeError:
        DF_module_enrich.loc[idx,'module_name'] = None

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

In [18]:
DF_pathway_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,pathway_name
0,5,map02010,1.081482e-08,2.433335e-06,0.500000,0.056738,0.101911,8.0,141.0,16.0,ABC transporters
1,5,map00920,4.686248e-07,5.272029e-05,0.250000,0.210526,0.228571,4.0,19.0,16.0,Sulfur metabolism
2,6,map00730,1.308624e-13,2.944404e-11,0.466667,0.333333,0.388889,7.0,21.0,15.0,Thiamine metabolism
3,6,map01100,4.889884e-04,5.501119e-02,0.533333,0.012678,0.024768,8.0,631.0,15.0,Metabolic pathways
4,8,map00270,3.797989e-22,8.545475e-20,0.428571,0.333333,0.375000,15.0,45.0,35.0,Cysteine and methionine metabolism


In [19]:
DF_module_enrich.head()

,imodulon,kegg_id,pvalue,qvalue,precision,recall,f1score,TP,target_set_size,imodulon_size,module_name
0,1,M00583,3.639319e-04,9.789769e-02,0.024096,1.000,0.047059,2.0,2.0,83.0,NaN
1,5,M00585,3.471665e-13,9.338779e-11,0.312500,1.000,0.476190,5.0,5.0,16.0,NaN
2,5,M00436,4.156064e-08,5.589906e-06,0.187500,1.000,0.315789,3.0,3.0,16.0,NaN
3,6,M00582,1.871400e-06,5.034067e-04,0.200000,0.375,0.260870,3.0,8.0,15.0,NaN
4,6,M00127,1.116177e-04,1.501258e-02,0.133333,0.400,0.200000,2.0,5.0,15.0,"Thiamine biosynthesis, prokaryotes, AIR (+ DXP..."


## SubtiWiki categories

In [39]:
DF_subtiwiki = pd.read_csv(path.join(external_data,'subtiwiki_categories.csv'))
DF_subtiwiki.head()

,SubtiWiki,BSU_number,FuncId,FuncName1,FuncName2,FuncName3,FuncName4,FuncName5
0,dat,BSU_09670,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
1,ddl,BSU_04560,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
2,glmM,BSU_01770,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
3,glmS,BSU_01780,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN
4,mraY,BSU_15190,SW 1.1,Cellular processes,cell envelope and cell division,cell wall synthesis,biosynthesis of peptidoglycan,NaN


In [40]:
# Change the subtiwiki annotation format into a list of genes and categories
DF_subtiwiki = DF_subtiwiki.rename({'BSU_number':'gene_id'},axis=1)
DF_subtiwiki = DF_subtiwiki.melt(id_vars='gene_id',value_vars=['FuncName1','FuncName2','FuncName3','FuncName4','FuncName5'])
DF_subtiwiki = DF_subtiwiki[DF_subtiwiki.value.notnull() & DF_subtiwiki.gene_id.isin(ica_data.gene_names)]
DF_subtiwiki.head()

,gene_id,variable,value
0,BSU_09670,FuncName1,Cellular processes
1,BSU_04560,FuncName1,Cellular processes
2,BSU_01770,FuncName1,Cellular processes
3,BSU_01780,FuncName1,Cellular processes
4,BSU_15190,FuncName1,Cellular processes


In [41]:
DF_subti_enrich = ica_data.compute_annotation_enrichment(DF_subtiwiki,'value')

## Save files

In [20]:
DF_GO_enrich['source'] = 'GO'
DF_pathway_enrich['source'] = 'KEGG pathways'
DF_module_enrich['source'] = 'KEGG modules'
# DF_subti_enrich['source'] = 'SubtiWiki'

DF_GO_enrich.rename({'gene_ontology':'annotation'},axis=1, inplace=True)
DF_pathway_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
DF_module_enrich.rename({'kegg_id':'annotation'},axis=1, inplace=True)
# DF_subti_enrich.rename({'value':'annotation'},axis=1, inplace=True)

#DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich, DF_subti_enrich])
DF_enrichments = pd.concat([DF_GO_enrich, DF_pathway_enrich, DF_module_enrich])
DF_enrichments.to_csv(path.join(erin_data_dir,'functional_enrichments.csv'))

# Check for single gene iModulons

Some iModulons are dominated by a single, high-coefficient gene. These iModulons may result from:
1. Overdecomposition of the dataset to identify noisy genes
1. Artificial knock-out of single genes
1. Regulons with only one target gene

No matter what causes these iModulons, it is important to be aware of them. The find_single_gene_imodulons function identifies iModulons that are likely dominated by a single gene.

The iModulons identified by ``find_single_gene_imodulons`` may contain more than one gene, since a threshold-agnostic method is used to identify these iModulons.

In [21]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)
len(sg_imods)

5

In [22]:
for i,mod in enumerate(sg_imods):
    ica_data.rename_imodulons({mod:'SG_'+str(i+1)})

In [23]:
ica_data.imodulon_table[ica_data.imodulon_table.single_gene == True]

,single_gene
SG_1,True
SG_2,True
SG_3,True
SG_4,True
SG_5,True


In [28]:
ica_data.view_imodulon('SG_5')

,gene_weight,gene_name,accession,old_locus_tag,start,end,strand,gene_product,COG,uniprot,operon
BSU_00970,0.603715,raeA,AL009126.3,BSU00970,116025,116537,+,ribosome-dependent mRNA endonuclease,Function unknown,P37574,Op59


# Save iModulon object

In [29]:
from pymodulon.util import explained_variance
from pymodulon.io import *

In [30]:
# Add iModulon sizes and explained variance
for im in ica_data.imodulon_names:
    ica_data.imodulon_table.loc[im,'imodulon_size'] = len(ica_data.view_imodulon(im))
    ica_data.imodulon_table.loc[im,'explained_variance'] = explained_variance(ica_data,imodulons=im)

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [32]:
save_to_json(ica_data, path.join('..','data','erin_test','bsu_raw.json.gz'))

If you prefer to view and edit your iModulon table in excel, save it as a CSV and reload the iModulon as before

In [31]:
ica_data.imodulon_table.to_csv(path.join('..','data','erin_test','imodulon_table_raw.csv'))

In [44]:
np.log2(100)

6.643856189774724

In [45]:
7.73-6.64

1.0900000000000007

In [46]:
np.log2(212/100)

1.0840642647884746

In [47]:
np.log2(212) - np.log2(100)

1.0840642647884744